<a href="https://colab.research.google.com/github/CS222-UIUC/team-109-project/blob/main/affanf2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install mysql-connector-python



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.1/34.1 MB 21.5 MB/s eta 0:00:00


In [2]:
!apt-get update
!apt-get install -y mysql-server
!service mysql start

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 https://cli.github.com/packages stable InRelease [3,917 B]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [2,149 kB]
Err:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages
  File has unexpected size (2147523 != 2148702). Mirror sync in progress? [IP: 2.16.244.11 443]
  Hashes of expected file:
   - Filesize:2148702 [weak]
   - SHA256:18305f8949ef5b7a8515c8ca4c9619fdbc6253c5fa028d5873502857f3c7da6a
   - SHA1:d52cabea17f65b952983e9de2101813f9ad4e58c [weak]
   - MD5Sum:ab1683212ddadd9211fe36c12ef0ddaf [weak]
  Release file created at: Tue, 18 Nov 2025 04:03:05 +0000
Get:6 https://cli.github.com/packages stable/main amd64 Packages [343 B]
Get:7 http://security.ubuntu.

In [3]:

!mysql -e "ALTER USER 'root'@'localhost' IDENTIFIED WITH mysql_native_password BY 'rootpass'; FLUSH PRIVILEGES;"
!mysql -uroot -prootpass -e "CREATE DATABASE my_database;"

mysql: [Warning] Using a password on the command line interface can be insecure.


In [4]:
import mysql.connector

conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="rootpass",
    database="my_database"
)
print("Connected!")

Connected!


In [5]:
cursor = conn.cursor()

# 2. Example SQL script
sql_script = """

DROP TABLE IF EXISTS state_gdp;
DROP TABLE IF EXISTS state_minimum_wage;
DROP TABLE IF EXISTS state_unemployment;
DROP TABLE IF EXISTS US_gdp;
DROP TABLE IF EXISTS US_minimum_wage;
DROP TABLE IF EXISTS US_unemployment;
DROP TABLE IF EXISTS President;

CREATE TABLE if not exists President  (
    number int primary key,
    name varchar(100),
    startTerm datetime unique,
    endTerm datetime unique
);

CREATE TABLE if not exists US_unemployment (
    timeSlot datetime primary key,
    value int,
    number int,
    foreign key(number) references President(number)
);
CREATE TABLE if not exists US_minimum_wage (
    timeSlot datetime primary key,
    value int,
    number int,
    foreign key(number) references President(number)
);
CREATE TABLE if not exists US_gdp (
    timeSlot datetime primary key,
    value int,
    number int,
    foreign key(number) references President(number)
);

CREATE TABLE if not exists state_unemployment (
    state varchar(13) not null,
    timeSlot datetime not null,
    value int,
    foreign key(timeSlot) references US_unemployment(timeSlot),
    primary key (state, timeSlot)
);
CREATE TABLE if not exists state_minimum_wage (
    state varchar(13) not null,
    timeSlot datetime not null,
    value int,
    foreign key(timeSlot) references US_minimum_wage(timeSlot),
    primary key (state, timeSlot)
);
CREATE TABLE if not exists state_gdp (
    state varchar(13) not null,
    timeSlot datetime not null,
    value int,
    foreign key(timeSlot) references US_gdp(timeSlot),
    primary key (state, timeSlot)
);

INSERT INTO President (number, name, startTerm, endTerm)
VALUES (16, 'Abraham Lincoln', '1861-3-4 00:00:00', '1865-4-15 00:00:00');

INSERT INTO US_unemployment (timeSlot, value, number)
VALUES ('2025-10-02 10:30:00', 20, 16),
       ('2025-10-02 10:31:00', 20, 16);

INSERT INTO state_unemployment (state, timeSlot, value)
VALUES ('Illinois', '2025-10-02 10:30:00', 20),
       ('Illinois', '2025-10-02 10:31:00', 20);

SELECT * FROM state_unemployment;
"""

# 3. Execute multiple SQL statements
for statement in sql_script.strip().split(";"):
    if statement.strip():  # Skip empty statements
        cursor.execute(statement)

# 4. Fetch results from last SELECT
rows = cursor.fetchall()
for row in rows:
    print(row)

# 5. Commit changes and close
conn.commit()
cursor.close()
conn.close()

('Illinois', datetime.datetime(2025, 10, 2, 10, 30), 20)
('Illinois', datetime.datetime(2025, 10, 2, 10, 31), 20)


In [6]:
import pandas as pd, mysql.connector

def run_select(sql):
    conn = mysql.connector.connect(host="localhost", user="root", password="rootpass", database="my_database")
    try:
        df = pd.read_sql(sql, conn)
    finally:
        conn.close()
    display(df)
    return df

q1 = """
SELECT su.state,su.timeSlot, AVG(su.value) as avg_state_unemp,(AVG(su.value) - AVG(nu.value)) as avg_spread, MAX(sg.value) as state_gdp
FROM state_unemployment su
JOIN US_unemployment nu
  ON nu.timeSlot = su.timeSlot
LEFT JOIN state_gdp sg
  ON sg.state = su.state AND sg.timeSlot = su.timeSlot
WHERE su.state = 'IL'
GROUP BY
  su.state,
  su.timeSlot
ORDER BY su.timeSlot
LIMIT 20;
"""

q2 = """
SELECT
  DATE_FORMAT(c.timeSlot, '%Y-%m') AS ym,
  c.state,
  c.metric,
  AVG(c.value) AS avg_state_metric,
  AVG(nu.value) AS avg_national_same_month
FROM (
  SELECT state, timeSlot, 'unemp' AS metric, value FROM state_unemployment
  UNION ALL
  SELECT state, timeSlot, 'minwage' AS metric, value FROM state_minimum_wage
) AS c
LEFT JOIN US_unemployment nu
  ON DATE_FORMAT(nu.timeSlot, '%Y-%m') = DATE_FORMAT(c.timeSlot, '%Y-%m')
WHERE c.state = 'Illinois'
GROUP BY DATE_FORMAT(c.timeSlot, '%Y-%m'), c.state, c.metric
ORDER BY ym DESC, c.metric
LIMIT 200;
"""

q3 = """
SELECT p.number, p.name, COUNT(DISTINCT u.timeSlot) as months_of_unemp,
  AVG(u.value) as avg_unemployment_during_term,
  COUNT(DISTINCT g.timeSlot) as quarters_of_gdp,
  AVG(g.value)as avg_gdp_during_term
FROM President p
LEFT JOIN US_unemployment u
  ON u.timeSlot BETWEEN p.startTerm AND p.endTerm
LEFT JOIN US_gdp g
  ON g.timeSlot BETWEEN p.startTerm AND p.endTerm
GROUP BY p.number, p.name
ORDER BY p.number
LIMIT 200;
"""

q4 = """
SELECT gq.yr,gq.qtr, gq.avg_gdp,
  (
    SELECT AVG(u.value)
    FROM US_unemployment u
    WHERE YEAR(u.timeSlot) = gq.yr
      AND QUARTER(u.timeSlot) = gq.qtr
  ) AS avg_unemp,
  gq.avg_gdp - (
    SELECT AVG(u2.value)
    FROM US_unemployment u2
    WHERE YEAR(u2.timeSlot) = gq.yr
      AND QUARTER(u2.timeSlot) = gq.qtr
  ) AS gdp_minus_unemp
FROM (
  SELECT
    YEAR(timeSlot) AS yr,
    QUARTER(timeSlot) AS qtr,
    AVG(value) AS avg_gdp
  FROM US_gdp
  GROUP BY YEAR(timeSlot), QUARTER(timeSlot)
) AS gq
ORDER BY gq.yr, gq.qtr
LIMIT 200;
"""

run_select(q1)
run_select(q2)
run_select(q3)
run_select(q4)

/tmp/ipython-input-3289456408.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, conn)


,state,timeSlot,avg_state_unemp,avg_spread,state_gdp


/tmp/ipython-input-3289456408.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, conn)


,ym,state,metric,avg_state_metric,avg_national_same_month
0,2025-10,Illinois,unemp,20.0,20.0


/tmp/ipython-input-3289456408.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, conn)


,number,name,months_of_unemp,avg_unemployment_during_term,quarters_of_gdp,avg_gdp_during_term
0,16,Abraham Lincoln,0,None,0,None


/tmp/ipython-input-3289456408.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, conn)


,yr,qtr,avg_gdp,avg_unemp,gdp_minus_unemp


,yr,qtr,avg_gdp,avg_unemp,gdp_minus_unemp


In [ ]:
con